# Week 3 GRAPPA Seminar Assignment

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

### Define the units

In [2]:
# length
cm = 1
m = 1e2 * cm
km = 1e3 * m
pc = 3.1e18 * cm
Mpc = 1e6 * pc
kpc = 1e3 * pc

# time
s = 1

# mass/energy
GeV = 1
eV = 1e-9 * GeV
kg = 5.61e26 * GeV
M_solar = 2e30 * kg
g = 1e-3 * kg


# other quantities
m_p = 0.938 * GeV
v0 = 220 * km/s
N_A = 6.022e23
rho0 = 0.3 * GeV/cm**3

In [3]:
# problem related quantities
A = 131.293 * g
m_det = 500 *  kg
T = 365*24*3600 * s
E_thresh = 3e3 * eV

factor_R0 = T * m_det * N_A * rho0 * A * v0 / m_p**2

In [11]:
def mu(m_x):
    m_N = A/N_A * GeV/g
    return m_x*m_N / (m_x + m_N)

def r(m_x):
    m_N = A/N_A * GeV/g
    return 4*m_x*m_N / (m_x + m_N)**2

def E0(m_x):
    return 0.5*m_x*v0**2

def sigma_px(m_x):
    return m_x * max_events * np.exp(E_thresh / (r(m_x) * E0(m_x))) / (factor_R0 * mu(m_x)**2)

In [13]:
confidence = 95
m_x = 100 * GeV

max_events = -np.log(1-confidence/100)


print(sigma_px(m_x)) ## in cm^2 ???

3.192633703505442e-42
